### uniprot

In [3]:
import requests
import json
import pandas as pd

In [3]:
df = pd.read_excel("../Data/AnalysisofMergedNetworkfromDrHarun.xlsx")
print(df.columns)
df.head(2)

NameError: name 'pd' is not defined

In [7]:
url = 'https://rest.uniprot.org/uniprotkb/search?query=P01023&format=json'
aaa = requests.get(url).json()
aaa

{'results': [{'entryType': 'UniProtKB reviewed (Swiss-Prot)',
   'primaryAccession': 'P01023',
   'secondaryAccessions': ['Q13677',
    'Q59F47',
    'Q5QTS0',
    'Q68DN2',
    'Q6PIY3',
    'Q6PN97'],
   'uniProtkbId': 'A2MG_HUMAN',
   'entryAudit': {'firstPublicDate': '1986-07-21',
    'lastAnnotationUpdateDate': '2024-01-24',
    'lastSequenceUpdateDate': '2010-10-05',
    'entryVersion': 242,
    'sequenceVersion': 3},
   'annotationScore': 5.0,
   'organism': {'scientificName': 'Homo sapiens',
    'commonName': 'Human',
    'taxonId': 9606,
    'lineage': ['Eukaryota',
     'Metazoa',
     'Chordata',
     'Craniata',
     'Vertebrata',
     'Euteleostomi',
     'Mammalia',
     'Eutheria',
     'Euarchontoglires',
     'Primates',
     'Haplorrhini',
     'Catarrhini',
     'Hominidae',
     'Homo']},
   'proteinExistence': '1: Evidence at protein level',
   'proteinDescription': {'recommendedName': {'fullName': {'value': 'Alpha-2-macroglobulin'},
     'shortNames': [{'value': '

In [8]:
with open('data.json', 'w') as f:
    json.dump(aaa, f)

In [ ]:
print(aaa["results"][0]["comments"][8]["texts"][0]["value"])

In [ ]:
for protein in protein_info:
    protein_name = protein['protein_name']
url = 'https://rest.uniprot.org/uniprotkb/search?query={protein_name} AND organism_id:9606&format=json'
response = requests.get(url)
data = response.json()
if data:
    text_value = data["results"][0]["comments"][8]["texts"][0]["value"]
    protein['Family'] = text_value
else:
    protein['Family'] = 'Not available' 
        
pd.DataFrame(protein_info)

1. similar proteins %100, %90, %50 (protein name, organism, length)
2. structure (Source, Identifier, Method, Resolution, Chain, Positions, Links)
3. variant

### disgenet +

In [4]:
import pandas as pd

df = pd.read_excel("../Data/AnalysisofMergedNetworkfromDrHarun.xlsx")
print(df.columns)
df.head(2)

Index(['Protein Name', 'ID', 'Class', 'Degree', 'Closeness', 'Eccentricity',
       'Clustering Coefficient', 'Stress', 'Neighborhood Connectivity',
       'Betweeness', 'Radiality', '# Undirected edges',
       'Topological Coefficeint', 'Nucleus', 'Plasma Membrane',
       'Mitochondrion', 'Extracellular', 'Target Family', 'Blood', 'Bone',
       'Bone Marrow', 'Cytoskeleton', 'Cytosol', 'Heart', 'Kidney',
       'Intestine', 'Liver', 'Muscle ', 'Saliva', 'Nervous System', 'Pancreas',
       'Intestine.1', 'Sequence', 'Family'],
      dtype='object')


,Protein Name,ID,Class,Degree,Closeness,Eccentricity,Clustering Coefficient,Stress,Neighborhood Connectivity,Betweeness,...,Kidney,Intestine,Liver,Muscle,Saliva,Nervous System,Pancreas,Intestine.1,Sequence,Family
0,A2M,P01023,Microtia,88,0.440871,5,0.379049,133916,196.363636,0.000349,...,4.097638,4.764426,4.984486,4.187754,2.771376,4.927944,3.638706,4.764426,MGKNKLLHPSLVLLLLVLLPTDASVSGKPQYMVLVPSLLHTETTEK...,Protein
1,A2ML1,Q96LQ8,Microtia,7,0.342512,5,0.333333,4616,100.857143,0.000024,...,1.104736,1.149553,1.204207,1.225949,1.503145,1.510835,0.783240,1.149553,MWAQLLLGMLALSPAIAEELPNYLVTLPARLNFPSVQKVCLDLSPG...,Protein


In [7]:
import requests
import json
import time

# Provide your API key 
API_KEY = "6092eaae-aca3-4933-9f2a-0b4098961b8d"

# Specify query parameters by means of a dictionary 
params = {}
# ...retrieve disease associated to gene with NCBI ID equal to 351 
params["gene_ncbi_id"] = "Q96LQ8"
# ...retrieve the first page of results (page number 0) 
params["page_number"] = "0"

# Create a dictionary with the HTTP headers of your API call 
HTTPheadersDict = {}
# Set the 'Authorization' HTTP header equal to API_KEY (your API key) 
HTTPheadersDict['Authorization'] = API_KEY
# Set the 'accept' HTTP header to specify the response format: one of 'application/json', 'application/xml', 'application/csv' 
HTTPheadersDict['accept'] = 'application/json'

# Query the gda summary endpoint 
response = requests.get("https://api.disgenetplus.com/api/v1/gda/summary",\
                        params=params, headers=HTTPheadersDict, verify=False)

# If the status code of response is 429, it means you have reached one of your query limits 
# You can retrieve the time you need to wait until doing a new query in the response headers 
if not response.ok:
    if response.status_code == 429:
        while response.ok is False:
            print("You have reached a query limit for your user. Please wait {} seconds until next query".format(\
            response.headers['x-rate-limit-retry-after-seconds']))
            time.sleep(int(response.headers['x-rate-limit-retry-after-seconds']))
            print("Your rate limit is now restored")

            # Repeat your query
            response = requests.get("https://api.disgenetplus.com/api/v1/gda/summary",\
                                    params=params, headers=HTTPheadersDict, verify=False)
            if response.ok is True:
                break
            else:
                continue

# Parse response content in JSON format since we set 'accept:application/json' as HTTP header 
response_parsed = json.loads(response.text)
print(response_parsed)
# print('STATUS: {}'.format(response_parsed["status"]))
# print('TOTAL NUMBER OF RESULTS: {}'.format(response_parsed["paging"]["totalElements"]))
# print('NUMBER OF RESULTS RETRIEVED BY CURRENT CALL (PAGE NUMBER {}): {}'.format(\
#       response_parsed["paging"]["currentPageNumber"], response_parsed["paging"]["totalElementsInPage"]))

c:\Python310\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.disgenetplus.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{'status': 'BAD_REQUEST', 'warnings': [], 'requestpar': {'gene_ncbi_id': 'Q96LQ8', 'page_number': 0}, 'userinfo': {'profile': 'TRIAL', 'profileDescription': 'Your DISGENET plus profile is TRIAL; you can access DISGENET plus REST endpoints with restrictions.'}, 'payload': {'details': 'Empty or invalid lists of both genes and disease identifiers.', 'message': '', 'timestamp': 'Wed May 01 19:21:44 UTC 2024'}}
